<div style="text-align:center;">
<h1>Partial Crawls</h1>
<h2>(Simplified -unofficial- version)</h2>
</div>

In [1]:
__author__ = "Lisette Espin-Noboa"
__reference__ = """
[1] Inference in OSNs via Lightweight Partial Crawls. Konstantin Avrachenkov, Bruno Ribeiro and Jithin K. Sreedharan
    https://github.com/jithin-k-sreedharan/HypRW
    https://hal.inria.fr/hal-01403018/document
    
[2] Stochastic Gradient Descent for Relational Logistic Regression via Partial Network Crawls
    https://arxiv.org/pdf/1707.07716.pdf

[3] Should We Be Confident in Peer Effects Estimated From Social Network Crawls?    
    https://www.aaai.org/ocs/index.php/ICWSM/ICWSM17/paper/viewFile/15696/14882
                """
__license__ = "GPL"
__version__ = "1.0.3"
__maintainer__ = "Lisette Espin-Noboa"
__email__ = "Lisette.Espin@gesis.org"
__status__ = "Developing"

In [2]:
###############################################################################################
# Warnings
###############################################################################################
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

###############################################################################################
# Dependencies
###############################################################################################
import numpy as np
import pandas as pd
import networkx as nx
from collections import Counter

In [3]:
###############################################################################################
# Functions
###############################################################################################

def random_walk_tour(g, n_tours, sn_size): 
    '''
    Performs a Lightweight partial crawl (see [1] and [3])
    Returns the super node, and the created tours.
    1.) A super node is created by selecting sn_size random nodes from g
    2.) A tour is created:
    2.1) A random walker starts from a randomly-selected node (seed) from the super node
    2.2) A random neighbor is selected and so-on.
    2.3) The random walker stops, if it finds a node from the super node.
    2.4) Repeat until gathering n_tours total tours.    
    '''
    np.random.seed(None)
    
    super_node = np.random.choice(list(g.nodes()),sn_size)
    tours = []
    
    while len(tours) < n_tours:        
        for seed in super_node:        
            tour = [seed]
            s = np.random.choice(g[seed],1)[0]

            while s not in super_node:
                tour.append(s)
                nbrs = g[s]
                if len(nbrs) > 0:
                    s = np.random.choice(nbrs,1)[0]
                else:
                    break

            tours.append(tour)
        
            if len(tours) >= n_tours:
                break
    
    return super_node, tours   

def get_priors(G, label, class_values, super_node, tours):
    size = len(class_values)
    n_tours = len(tours)
    prior = pd.Series(np.zeros((size)), index=class_values)

    ds = sum([G.degree(s) for s in super_node])
    m = n_tours
    for c in class_values:
        sum_tours = sum([1/G.degree(v) for tour in tours for v in tour[1:] if G.node[v][label] == c])
        sum_super_node = sum([1 for v in super_node if G.node[v][label] == c])
        prior.loc[c] = ((ds/m) * sum_tours) + sum_super_node
        
    prior += 1  # smoothing
    return prior / prior.sum() # normalizing

def get_conditional_probabilities(G, label, class_values, super_node, tours):
    columns = ['{}-{}'.format(l, l) for l in class_values]
    cpn = {label: pd.DataFrame(np.zeros((len(class_values), len(columns))), index=class_values, columns=columns)}

    n_tours = len(tours)
    isu = not nx.is_directed(G)
    condprobs = np.zeros((len(class_values), len(class_values)))
    
    ds = sum([G.degree(s) for s in super_node])
    m = n_tours
    
    subgraph = G.subgraph(list(set(super_node) | set([node for tour in tours for node in tour])))
        
    for row, a in enumerate(class_values):  # ego class (condition)
        for col, b in enumerate(class_values):  # neighbor class

            sum_tours = sum([1 for tour in tours for t,v in enumerate(tour) if t >= 2 and G.node[tour[t-1]][label]==a and G.node[v][label]==b])
            sum_super_node = sum([1 for edge in subgraph.edges() if edge[0] in super_node or edge[1] in super_node])
            tmp = ((ds/m)*sum_tours) + sum_super_node
            
            cpn[label].loc[a, '{}-{}'.format(b, b)] += tmp

            if isu and a != b:
                # undirected and heterophilic edge then symmetric
                cpn[label].loc[b, '{}-{}'.format(a, a)] += tmp

    cpn[label] += 1  # smoothing
    cpn[label] = cpn[label].div(cpn[label].sum(axis=1), axis=0) # normalizing
    return cpn

def get_class_priors(G, label, class_values, super_node, tours):    
    '''
    Returns the class prior estimates.
    - P(m) probability of being a minority (class label m)
    - P(M) probability of being a majority (class label M)
    - P(m) + P(M) = 1    
    
    See Eq. 2 from [3].
    '''
    size = len(class_values)
    n_tours = len(tours)    
    prior = np.zeros(len(class_values))

    ds = sum([G.degree(s) for s in super_node])
    m = n_tours
    for i,c in enumerate(class_values):
        sum_tours = sum([1/G.degree(v) for tour in tours for v in tour[1:] if G.node[v][label] == c])
        sum_super_node = sum([1 for v in super_node if G.node[v][label] == c])
        prior[i] = ((ds/m) * sum_tours) + sum_super_node
        
    prior += 1  # smoothing
    return prior / prior.sum() # normalizing

def get_conditional_probabilities(G, label, class_values, super_node, tours):
    '''
    Returns the conditional probability estimates.
    - P(m|m) probability of ego being a minority given that ego is connected to a minorty
    - P(m|M) probability of ego being a minority given that ego is connected to a majority
    - P(M|m) probability of ego being a majority given that ego is connected to a minorty
    - P(M|M) probability of ego being a majority given that ego is connected to a majority    
    - P(m|*) = 1
    - P(M|*) = 1
    
    See Eq. 3, 4 from [3].
    '''
    n_tours = len(tours)
    isu = not nx.is_directed(G)
    condprobs = np.zeros((len(values),len(values)))    
    
    ds = sum([G.degree(s) for s in super_node])
    m = n_tours
    
    subgraph = G.subgraph(list(set(super_node) | set([node for tour in tours for node in tour])))
        
    for i, a in enumerate(class_values):  # ego class (condition)
        for j, b in enumerate(class_values):  # neighbor class

            sum_tours = sum([1 for tour in tours for t,v in enumerate(tour) if t >= 2 and G.node[tour[t-1]][label]==a and G.node[v][label]==b])
            sum_super_node = sum([1 for edge in subgraph.edges() if (edge[0] in super_node or edge[1] in super_node) and (G.node[edge[0]][label]==a and G.node[edge[1]][label]==b) ])
            tmp = ((ds/m)*sum_tours) + sum_super_node
            
            condprobs[i,j] += tmp

            if isu and a != b:
                # undirected and heterophilic edge then symmetric
                condprobs[j,i] += tmp

    condprobs += 1  # smoothing    
    return condprobs / condprobs.sum(axis=0)[:,None] # normalizing
    

<h2>Toy-Example</h2>

<h3>1. Graph</h3>

In [4]:
nnodes = 2000
minedges = 2
G = nx.barabasi_albert_graph(n=nnodes, m=minedges, seed=None)
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 2000
Number of edges: 3996
Average degree:   3.9960


<h3>2. Node attributes</h3>

In [5]:
minority = int(round(G.number_of_nodes() * 10 / 100))
minorities = np.random.choice(G.nodes(),minority)
attributes = {n:'m' if n in minorities else 'M' for n in G.nodes()}
label = 'group'
values = ['M','m']
nx.set_node_attributes(G,name=label,values=attributes)

<h3>3. Partial Crawls</h3>

In [6]:
sn_size = int(round(G.number_of_nodes() * 10 / 100))
n_tours = int(sn_size*0.5)

super_node, tours = random_walk_tour(G, n_tours, sn_size)
print('- super_node size: {}'.format(len(super_node)))
print('- No. tours: {}'.format(len(tours)))
n_crawled_nodes = len(set([n for tour in tours for n in tour]))
print('- No. unique crawled nodes: {} ({}%)'.format(n_crawled_nodes,round(n_crawled_nodes*100/G.number_of_nodes(),0)))
tour_sizes = [len(tour) for tour in tours]
print('- Average tour size: {}'.format(np.mean(tour_sizes)))
print('- Min tour size: {}'.format(min(tour_sizes)))
print('- Max tour size: {}'.format(max(tour_sizes)))

- super_node size: 200
- No. tours: 100
- No. unique crawled nodes: 543 (27.0%)
- Average tour size: 9.51
- Min tour size: 1
- Max tour size: 62


<h3>4. Estimates</h3>

In [7]:
tmp = get_class_priors(G, label, values, super_node, tours)
pd.Series(tmp, index=values)

M    0.898206
m    0.101794
dtype: float64

In [8]:
tmp = get_conditional_probabilities(G, label, values, super_node, tours)
pd.DataFrame(tmp, index=values, columns=values)

,M,m
M,0.812234,0.187766
m,0.959573,0.040427


<h3>5. Empirical</h3>

In [9]:
tmp = Counter([a for n,a in attributes.items()])
pd.Series([tmp[a] for a in values], index=values) / G.number_of_nodes()

M    0.906
m    0.094
dtype: float64

In [10]:
tmp = np.zeros((len(values),len(values)))
counts = Counter(['{}-{}'.format(G.node[e[0]][label],G.node[e[1]][label]) for e in G.edges()])
isu = not nx.is_directed(G)

for r,v1 in enumerate(values):
    for c,v2 in enumerate(values):
        ec = counts['{}-{}'.format(v1,v2)]
        tmp[r,c] += ec
        
        if isu and r!=c:
            tmp[c,r] += ec
    
tmp = tmp/tmp.sum(axis=1, keepdims=True)
pd.DataFrame(tmp, index=values, columns=values)

,M,m
M,0.847936,0.152064
m,0.961783,0.038217
